In [1]:
from IPython.display import clear_output
from functions import *
import numpy as np
import pandas as pd
from transformers import *
import tensorflow as tf
import pickle
import scipy as sc
import math as mt
import random
# from scipy import cluster as clst
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.decomposition import PCA

## Import data

In [104]:
#dfd = pd.read_csv('sts-dev.csv', delimiter='\t' , error_bad_lines=False)
#STS-B
dfd = pd.read_csv('sts-test.csv', delimiter='\t' , error_bad_lines=False)
#STS-2012
#dfd = pd.read_csv('../data/sts-2012-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2013
#dfd = pd.read_csv('../data/sts-2013-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2014
#dfd = pd.read_csv('../data/sts-2014-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#STS-2015
#dfd = pd.read_csv('../data/sts-2015-test/STS.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
#dfd = dfd[dfd["corr"] != -1]
#STS-2016
# dfd = pd.read_csv('../data/sts-2016-test/STS2016.input.ALL.csv', delimiter='\t' , error_bad_lines=False)
# dfd = dfd[dfd["corr"] != -1]
#SICK-R
#dfd = pd.read_csv('../data/sick-r-test/SICK.csv', delimiter='\t' , error_bad_lines=False)
clear_output()

In [105]:
dfd

,type,subtype,year,num,corr,sentence1,sentence2
0,main-captions,MSRvid,2012test,24,2.5,A girl is styling her hair.,A girl is brushing her hair.
1,main-captions,MSRvid,2012test,33,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.
2,main-captions,MSRvid,2012test,45,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.
3,main-captions,MSRvid,2012test,63,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.
4,main-captions,MSRvid,2012test,66,1.5,A man is playing a harp.,A man is playing a keyboard.
...,...,...,...,...,...,...,...
1090,main-news,headlines,2015,1438,0.4,"US, China fail to paper over cracks in ties",China: Relief in focus as hope for missing fades
1091,main-news,headlines,2015,1454,1.4,World Cup live: France 0-0 Germany,World Cup live: Germany 0-0 Ghana
1092,main-news,headlines,2015,1456,4.8,Tokyo to host 2020 Games,Tokyo wins race to host 2020 Olympic Games
1093,main-news,headlines,2015,1463,4.4,France warns of extremists benefiting from Egy...,France fears extremists will benefit from Egyp...


In [106]:
# Check for errors
dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
len(dfd)

1095

## Loading models - choose and run one cell

In [5]:
# BERT
casing = "bert-base-uncased" 
tokenizer = BertTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)

config = BertConfig(dropout=0.2, attention_dropout=0.2 ) #hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2
config.output_hidden_states = True

model = TFBertModel.from_pretrained(casing, config = config)
model.trainable = False
emb_len = 768
clear_output()

# BERT
n_cluster = 27 # Number of clusters to use
n_pc = 12 # Number of main principal components to drop for local method
n_pc_global = 15 # Number of main principal components to drop for global method

In [5]:
# GPT-2
casing = "gpt2" 
tokenizer = GPT2Tokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = GPT2Config()
config.output_hidden_states = True

model = TFGPT2Model.from_pretrained(casing, config=config)
model.trainable = False

emb_len = 768
clear_output()

# GPT2
n_cluster = 10
n_pc = 30
n_pc_global = 30

In [54]:
# RoBERTa
casing = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)
config = RobertaConfig.from_pretrained(casing)
config.output_hidden_states = True

model = TFRobertaModel.from_pretrained(casing, config=config)
model.trainable = False
emb_len = 768
clear_output()

# RoBERTa
n_cluster = 27
n_pc = 12
n_pc_global = 25

## Get sentences and words representations

In [107]:
# Get sentence representations BERT/GPT2/RoBERTa
#sentences = get_representations(dfd, tokenizer, model, emb_len)
sentences = np.load("roberta-stsb-test.npy", allow_pickle=True).tolist()
words = getWords(sentences)
clear_output()

In [108]:
len(sentences)//2

1095

In [33]:
# HELPER
np.save("gpt2-sick-test.npy", np.asarray(sentences, dtype=object))

### Run multiple experiments...

In [7]:
# datasets = [ "sts-test.csv", "../data/sts-2015-test/STS.input.ALL.csv", "../data/sts-2014-test/STS.input.ALL.csv", "../data/sts-2013-test/STS.input.ALL.csv", "../data/sts-2012-test/STS.input.ALL.csv"]
# npynames = ["gpt2-stsb-test.npy", "gpt2-sts2015-test.npy", "gpt2-sts2014-test.npy", "gpt2-sts2013-test.npy", "gpt2-sts2012-test.npy" ]

# for i in range(len(datasets)):
#     dsName = datasets[i]
#     npyName = npynames[i]
#     dfd = pd.read_csv(dsName, delimiter='\t' , error_bad_lines=False)
#     dfd = dfd[dfd["corr"] != -1]
#     dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
#     sentences = get_representations(dfd, tokenizer, model, emb_len)
#     np.save(npyName, np.asarray(sentences, dtype=object))


## Get representations via baseline, global method, local (isotropy enhancing) method 

In [109]:
# BASELINE
baseline_sentence_rep = mean_pooling(words, sentences)
baseline_score = similarity(baseline_sentence_rep, emb_len)

# GLOBAL METHOD
global_representations = global_method(np.asarray(words), n_pc_global, emb_len)
global_sentence_rep = mean_pooling(global_representations, sentences)
global_score = similarity(global_sentence_rep, emb_len)

# LOCAL METHOD
#isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
# calculating sentence representations based on mean word (isotropised) representations
#sentence_rep = mean_pooling(isotropic_representations, sentences)
#local_score = similarity(sentence_rep, emb_len)

## BOOTSTRAP

In [110]:
means_global = []
means_local = []
means_baseline = []
all_scores_local = []
stds_local = []
stds_global = []
stds_baseline = []
scores_local_full = []
for j in range(5):
    coeffs_global = []
    coeffs_local = []
    coeffs_baseline = []
    # LOCAL METHOD
    isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
    sentence_rep = mean_pooling(isotropic_representations, sentences)
    score_full = similarity(sentence_rep, emb_len)
    scores_local_full.append(sper_corrcoef(dfd["corr"].tolist(), score_full))
    # Run k bootstrap samples to estimate error 
    for k in range(40):
        ixs = random.choices(list(range(len(sentences)//2)),k=len(sentences))
        baseline_bootstrap = []
        global_bootstrap = []
        local_bootstrap = []
        scores_bootstrap = []
        for i in ixs:
            if j == 0:
                baseline_bootstrap.append(baseline_sentence_rep[2*i])
                global_bootstrap.append(global_sentence_rep[2*i])
                baseline_bootstrap.append(baseline_sentence_rep[2*i+1])
                global_bootstrap.append(global_sentence_rep[2*i+1])
            local_bootstrap.append(sentence_rep[2*i])
            local_bootstrap.append(sentence_rep[2*i+1])
            scores_bootstrap.append(dfd['corr'].iloc[i])

        if j == 0:
            global_scoore_bs = similarity(global_bootstrap, emb_len)
            baseline_scoore_bs = similarity(baseline_bootstrap, emb_len)
        local_scoore_bs = similarity(local_bootstrap, emb_len)

        if j == 0:
            coeffs_baseline.append(sper_corrcoef(scores_bootstrap, baseline_scoore_bs))
            coeffs_global.append(sper_corrcoef(scores_bootstrap, global_scoore_bs))
        coeffs_local.append(sper_corrcoef(scores_bootstrap, local_scoore_bs))


    all_scores_local += coeffs_local
    if j == 0:
        means_baseline.append(np.mean(coeffs_baseline))
        means_global.append(np.mean(coeffs_global))
        stds_global.append(np.std(coeffs_global))
        stds_baseline.append(np.std(coeffs_baseline))

    means_local.append(np.mean(coeffs_local))
    stds_local.append(np.std(coeffs_local))
    
    print("run ",j,", means and errors:")
    if j == 0:
        print("baseline:",np.mean(coeffs_baseline),"+-",np.std(coeffs_baseline))
        print("global:",np.mean(coeffs_global),"+-",np.std(coeffs_global))
    print("local:",np.mean(coeffs_local),"+-",np.std(coeffs_local))


run  4 , means and errors:
local: 72.2106085872343 +- 1.1794197221874692


In [111]:
for i in range(len(scores_local_full)):
    print(scores_local_full[i],"+-",stds_local[i])

71.65948633320005 +- 1.0979745194922623
71.9418373610387 +- 1.271562268471421
71.78731463664705 +- 0.9968744583662337
71.4765840956824 +- 1.1268121289716495
71.819495553112 +- 1.1794197221874692


In [112]:
print("baseline bs std: +-",stds_baseline[0])
print("global bs std: +-",stds_global[0])

baseline bs std: +- 1.4412689453310223
global bs std: +- 1.1140048742865518


In [113]:
print("scores_local_full-mean:", np.mean(scores_local_full))
print("scores_local_full-std:", np.std(scores_local_full))
print("mean_scores_local_bs-std:", np.std(means_local))
print("all_scores_local_bs-std:", np.std(all_scores_local))

scores_local_full-mean: 71.73694359593603
scores_local_full-std: 0.15818989808247108
mean_scores_local_bs-std: 0.3714824887731119
all_scores_local_bs-std: 1.1972393232938126


In [55]:
# performance
print("Spearman Correlation baseline: ",sper_corrcoef(dfd['corr'], baseline_score))
print("Spearman Correlation global: ",sper_corrcoef(dfd['corr'], global_score))
print("Spearman Correlation local: ",sper_corrcoef(dfd['corr'], local_score))

# isotropy of space
print("Isotropy baseline: ", isotropy(np.asarray(baseline_sentence_rep, dtype=np.float64)))
print("Isotropy global: ", isotropy(global_representations))
print("Isotropy local: ", isotropy(isotropic_representations))

Spearman Correlation baseline:  48.11715446651223
Spearman Correlation global:  64.38035591129334
Spearman Correlation local:  67.63985006690713
Isotropy baseline:  0.0001104029659049792
Isotropy global:  0.5206524086147406
Isotropy local:  0.7535314129175708


In [48]:
# REPEAT LAST METHOD
isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
sentence_rep = mean_pooling(isotropic_representations, sentences)
local_score = similarity(sentence_rep, emb_len)

print("Spearman Correlation local: ",sper_corrcoef(dfd['corr'], local_score))
#print("Isotropy local: ", isotropy(isotropic_representations))

Spearman Correlation local:  65.07141874721928


### Run multiple experiments...

In [5]:
# npynames = ["roberta-stsb-test.npy", "roberta-sts2015-test.npy", "roberta-sts2014-test.npy", "roberta-sts2013-test.npy", "roberta-sts2012-test.npy" ]
# datasets = [ "sts-test.csv", "../data/sts-2015-test/STS.input.ALL.csv", "../data/sts-2014-test/STS.input.ALL.csv", "../data/sts-2013-test/STS.input.ALL.csv", "../data/sts-2012-test/STS.input.ALL.csv"]
# appendfile = open("results3.txt", "a")

# for i in range(len(npynames)):
#     dsName = datasets[i]
#     dfd = pd.read_csv(dsName, delimiter='\t' , error_bad_lines=False)
#     dfd = dfd[dfd["corr"] != -1]
#     dfd = dfd[np.logical_not(pd.isna(dfd["sentence2"]))]
#     sentences = np.load(npynames[i], allow_pickle=True).tolist()
#     words = getWords(sentences)

#     baseline_sentence_rep = mean_pooling(words, sentences)
#     baseline_score = similarity(baseline_sentence_rep, emb_len)

#     # GLOBAL METHOD
#     global_representations = global_method(np.asarray(words), n_pc_global, emb_len)
#     global_sentence_rep = mean_pooling(global_representations, sentences)
#     global_score = similarity(global_sentence_rep, emb_len)

#     # LOCAL METHOD
#     isotropic_representations = cluster_based(np.asarray(words), n_cluster, n_pc, emb_len)
#     # calculating sentence representations based on mean word (isotropised) representations
#     sentence_rep = mean_pooling(isotropic_representations, sentences)
#     local_score = similarity(sentence_rep, emb_len)

#     appendfile.write(npynames[i]+"\n")
#     appendfile.write("Spearman Correlation baseline: " + str(sper_corrcoef(dfd['corr'], baseline_score)) + "\n")
#     appendfile.write("Spearman Correlation global: " + str(sper_corrcoef(dfd['corr'], global_score))+ "\n")
#     appendfile.write("Spearman Correlation local: " + str(sper_corrcoef(dfd['corr'], local_score))+ "\n")
#     appendfile.flush()
#     # isotropy of space
#     appendfile.write("Isotropy baseline: " + str(isotropy(np.asarray(baseline_sentence_rep, dtype=np.float64)))+ "\n")
#     appendfile.write("Isotropy global: " + str(isotropy(global_representations))+ "\n")
#     appendfile.write("Isotropy local: " + str(isotropy(isotropic_representations))+ "\n")
#     appendfile.write("---------\n")
#     appendfile.flush()